# pyLectureMultiModalAnalysis

## Feature extraction from video segment

### Functions: 2
#### video2frame(), frame2features()

### Author: Stelios Karozis

## RandomVector()

In [51]:
def RandomVector(trainmode=True,sz=100):
    import pickle
    import numpy as np
    from numpy import array
    import random
    
    if trainmode==False:
        try:
            vector = pickle.load(open("random.pickle", "rb"))
        except (OSError, IOError) as e:
            vector = 3
            print('Error the random vector is missing')
    else:
        vector=[]
        for i in range(sz):
            tmp=random.uniform(0.5, 1.0)
            vector.append(tmp)
    with open('random.pickle','wb') as f:
        pickle.dump(array(vector), f)
    return array(vector)

## video2frame()

In [30]:
def video2frame(count,sec,folderVID,file,folderIMG):
    import cv2
    vidcap = cv2.VideoCapture(folderVID+file)
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(folderIMG+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames,folderIMG+str(count)+".jpg"   #folderIMG+file+'_'+str(count)+".jpg"

### test

In [ ]:
file='hd0456.mov'
sec = 0
frameRate = 0.5 #//it will capture image in each 0.5 second -> 2fps
count=1
success,a = video2frame(sec,'./',file,'./')
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success,a = video2frame(sec,'./',file,'./')

## frame2features

In [81]:
def frame2features(frame):
    import tensorflow as tf
    from keras.preprocessing import image
    from keras.applications.vgg16 import VGG16
    from keras.applications.vgg16 import preprocess_input
    import numpy as np

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

    model = VGG16(weights='imagenet', include_top=False)
    #model.summary()

    fr_path = frame
    fr = image.load_img(fr_path, target_size=(224, 224))
    fr_data = image.img_to_array(fr)
    fr_data = np.expand_dims(fr_data, axis=0)
    fr_data = preprocess_input(fr_data)

    vgg16_feature = model.predict(fr_data)

    vgg16_feature_np = np.array(vgg16_feature)
    feature = vgg16_feature_np.flatten()
    tf.reset_default_graph()
    vector = RandomVector(trainmode=True,sz=100)
    return np.dot(feature[:,None],vector[None,:])

### test

In [ ]:
frame2features('1.jpg')

## Video2feature()

In [83]:
def Video2feature(pathIn='./data/',frameRate=0.5, save=True):
    import os
    import glob
    import numpy as np
    import pandas as pd
    import pathlib
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

    pathIn=pathIn
    index = pd.read_csv(pathIn+'index.csv', sep=';')

    frameRate = frameRate #//it will capture image in each 0.5 second -> 2fps
    ftr_array=[]

    for f,s in index[['FILE','SEG']].values:
      
        pathOut=pathIn+f+'/frames/'
        if not os.path.exists(pathOut):
            os.makedirs(pathOut)
     
        files=f+'/'+str(s)   
        files=glob.glob(pathIn+files+'*')[0]
        suffix=os.path.splitext(files)[1]
        files=f+'/'+str(s)+suffix
        ftr_fr=[]
        filename=pathIn + files
        print(filename)
        sec = 0
        count=1
        success,fr = video2frame(count,sec,pathIn,files,pathOut)
        print(fr)
        ftr = frame2features(fr)
        ftr_fr.append(ftr)

        while success:
            count = count + 1
            sec = sec + frameRate
            sec = round(sec, 2)
            success,fr = video2frame(count,sec,pathIn,files,pathOut)
            if success == True:
                print(fr)
                ftr = frame2features(fr)
                ftr_fr.append(ftr)
        ftr_fr = np.vstack(ftr_fr) 
        ftr_fr = np.average(ftr_fr, axis=0)
        ftr_array.append(ftr_fr)
        
    ftr_array = np.vstack(ftr_array)


    ftr_df = pd.DataFrame(data=ftr_array)
    df=index.copy()
    df=pd.concat([df,ftr_df], axis=1)

    if save == True:
        df.to_pickle(pathIn+'Video2Features.pkl')

    return df

In [85]:
df = Video2feature(pathIn='../data/',frameRate=0.5, save=False)
df

../data/medium_0/part_0.mp4
../data/medium_0/frames/1.jpg
../data/medium_0/frames/2.jpg
../data/medium_0/frames/3.jpg
../data/medium_0/frames/4.jpg
../data/medium_0/frames/5.jpg
../data/medium_0/frames/6.jpg
../data/medium_0/frames/7.jpg
../data/medium_0/frames/8.jpg
../data/medium_0/frames/9.jpg
../data/medium_0/frames/10.jpg
../data/medium_0/frames/11.jpg
../data/medium_0/frames/12.jpg
../data/medium_0/frames/13.jpg
../data/medium_0/frames/14.jpg
../data/medium_0/frames/15.jpg
../data/medium_0/frames/16.jpg
../data/medium_0/frames/17.jpg
../data/medium_0/frames/18.jpg
../data/medium_0/frames/19.jpg
../data/medium_0/frames/20.jpg
../data/medium_0/frames/21.jpg
../data/medium_0/frames/22.jpg
../data/medium_0/frames/23.jpg
../data/medium_0/frames/24.jpg
../data/medium_0/frames/25.jpg
../data/medium_0/frames/26.jpg
../data/medium_0/frames/27.jpg
../data/medium_0/frames/28.jpg
../data/medium_0/frames/29.jpg
../data/medium_0/frames/30.jpg
../data/medium_0/frames/31.jpg
../data/medium_0/fra

,FILE,SEG,CLASS_1,0,1,2,3,4,5,6,...,90,91,92,93,94,95,96,97,98,99
0,medium_0,part_0,boring,0.956490,0.932271,0.987480,0.987353,0.915322,0.964926,0.935252,...,0.905785,0.999660,0.922155,0.994583,0.873916,0.948770,0.939730,0.937662,0.942870,0.987104
1,medium_7,part_28,boring,1.800701,1.851419,1.902076,1.831315,1.808389,1.786130,1.837504,...,1.732694,1.839001,1.775356,1.804544,1.820373,1.819947,1.849141,1.918971,1.923281,1.798236
2,medium_4,part_12,boring,0.981370,1.047871,1.024990,0.940043,1.000307,1.004285,0.988809,...,1.020559,1.007974,1.008597,0.972289,1.014508,0.990719,0.977606,0.906477,0.999844,0.933708
